# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/peft/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/peft/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦象征着天、强、积极、刚健、行动、刚毅等。乾卦由六个阳爻夹一个阴爻构成，形状为阳刚之卦。

乾卦的含义非常丰富，它不仅代表天、强、积极、刚健等特质，还象征着行动力、决断力、领导力等。在《易经》中，乾卦所代表的能量是不断向上、向右发展的，象征着事物的发展和进步。

乾卦还有一个重要的象征意义，就是“天行健”，意味着天地之间的事情都在不断地发展变化着，顺应自然规律，就能获得成功。因此，乾卦也教导我们要尊重自然、顺应自然，才能取得成功。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

周易中的讼卦（又称法卦）是《易经》中的第五卦，是一个阳刚之卦。讼卦由两个阴爻夹一个阳爻构成，形状为上阴下阳。这个卦象象征着诉讼、争端、诉讼案件等。

讼卦的含义非常丰富，它不仅代表诉讼、争端等负面事物，还意味着通过诉讼、争端等手段来解决问题。因此，讼卦教育我们要有决断力、勇气和毅力，才能够解决纷争和问题。

此外，讼卦还有一个重要的象征意义，就是“天听”，意味着天地之间的事情都在不断地发展变化着，顺应自然规律，就能获得成功。因此，讼卦也教导我们要尊重自然、顺应自然，才能取得成功。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [10]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240519_233402"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [11]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [12]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》六十四卦之首，象征天，代表着刚强、健行和大通。这个卦象预示着吉祥如意，提醒人们遵守天道的德行。在运势和事业方面，乾卦暗示着名利双收，但也需小心谨慎，警惕力戒骄傲。在经商方面，此卦预示着发展向上的大好机会，但宜冷静分析形势，坚持商业道德，并对困难冷静应对。在求名方面，乾卦的建议是积累知识、坚定信念、自强不息，成君子之名。在婚恋方面，阳盛阴衰，男性宜找温柔之人，女性温柔者更佳。在决策上，乾卦建议人们坚持刚健、正直、公允的实质，修养德行，必能克服困难，消除灾难。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240519_233402）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是《易经》六十四卦之首，象征天，代表着刚强、健行和大通。这个卦象预示着吉祥如意，提醒人们遵守天道的德行。在运势和事业方面，乾卦暗示着名利双收，但也需小心谨慎，警惕力戒骄傲。在经商方面，此卦预示着发展向上的大好机会，但宜冷静分析形势，坚持商业道德，并对困难冷静应对。在求名方面，乾卦的建议是积累知识、坚定信念、自强不息，成君子之名。在婚恋方面，阳盛阴衰，男性宜找温柔之人，女性温柔者更佳。在决策上，乾卦建议人们坚持刚健、正直、公允的实质，修养德行，必能克服困难，消除灾难。


In [13]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是一种充满争议和危险的卦象，在其卦辞和象辞中都暗示了相互之间的对抗。然而，此卦也提醒人们，争端的产生可以成为推动事业发展的一种力量。在谋事之初，必须慎之又慎，不宜执拗固执。卜得此爻，遇到诉讼等事宜，应该尽量回避，若未能如愿，切勿争端不休。只有坚持正义，以和为贵，原则铁面，才能迎来好结果。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240519_233402）：
[gMASK]sop 周易中的讼卦是什么卦象，讼卦是一种充满争议和危险的卦象，在其卦辞和象辞中都暗示了相互之间的对抗。然而，此卦也提醒人们，争端的产生可以成为推动事业发展的一种力量。在谋事之初，必须慎之又慎，不宜执拗固执。卜得此爻，遇到诉讼等事宜，应该尽量回避，若未能如愿，切勿争端不休。只有坚持正义，以和为贵，原则铁面，才能迎来好结果。


In [ ]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [ ]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [ ]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [ ]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [ ]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

In [ ]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)